# Tutorial 11: Competitive Intelligence Workflow with DevUI

## Overview

This tutorial demonstrates how to build a **production-ready multi-agent workflow** for competitive intelligence analysis using:

-  **Sequential Workflow**: Chain multiple agents using `SequentialBuilder`
-  **DevUI Integration**: Visual workflow execution and monitoring
-  **Azure AI Agent File Search**: Extract data from PDF catalogs
-  **Multi-Agent Collaboration**: Data flows from one agent to the next

### Workflow Architecture

```
Agent 1 (Data Extraction) → Agent 2 (Pricing Analysis) → 
Agent 3 (Visualization) → Agent 4 (Report Generation) → Final Report
```

## Step 1: Install Dependencies

In [24]:
import sys
import subprocess

packages = [
    'python-dotenv',
    'matplotlib',
    'seaborn',
    'pandas',
    'numpy',
    'agent-framework',
    'agent-framework-devui --pre',
]

for package in packages:
    pkg_name = package.split()[0]  # Handle --pre flag
    try:
        __import__(pkg_name.replace('-', '_'))
        print(f" {pkg_name} already installed")
    except ImportError:
        print(f" Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install"] + package.split())
        print(f" {pkg_name} installed")

print("\n All dependencies ready!")

 Installing python-dotenv...
 python-dotenv installed
 matplotlib already installed
 seaborn already installed
 pandas already installed
 numpy already installed
 agent-framework already installed
 agent-framework-devui already installed

 All dependencies ready!
 python-dotenv installed
 matplotlib already installed
 seaborn already installed
 pandas already installed
 numpy already installed
 agent-framework already installed
 agent-framework-devui already installed

 All dependencies ready!


## Step 2: Import Libraries

In [25]:
import asyncio
import json
import os
from pathlib import Path
from typing import Any, Dict, List, Optional
from datetime import datetime

# Data analysis and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Agent framework
from agent_framework import (
    ChatAgent,
    ChatMessage,
    Executor,
    HostedFileSearchTool,
    HostedVectorStoreContent,
    Role,
    SequentialBuilder,
    WorkflowContext,
    handler,
)
from agent_framework_azure_ai import AzureAIAgentClient
from azure.ai.agents.models import FileInfo, VectorStore
from azure.identity.aio import AzureCliCredential
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set plot style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print(" Imports successful!")
print(" Ready for workflow creation!")

 Imports successful!
 Ready for workflow creation!


## Step 3: Setup Environment

In [26]:
# Create folder structure
FOLDERS = {
    'input': './competitive_analysis/input',
    'output': './competitive_analysis/output',
    'data': './competitive_analysis/data',
    'charts': './competitive_analysis/charts',
}

for folder_name, folder_path in FOLDERS.items():
    Path(folder_path).mkdir(parents=True, exist_ok=True)
    print(f" {folder_name}: {folder_path}")

# Verify configuration
project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
if not project_endpoint:
    raise ValueError(" AZURE_AI_PROJECT_ENDPOINT not set in .env file")

print(f"\n Azure AI Project: {project_endpoint}")
print(" Environment ready!")

 input: ./competitive_analysis/input
 output: ./competitive_analysis/output
 data: ./competitive_analysis/data
 charts: ./competitive_analysis/charts

 Azure AI Project: https://gk-agent-framework-project.services.ai.azure.com/api/projects/agentframworkProject
 Environment ready!


## Step 4: Build Agent 1 - Data Extraction Executor

This executor uses Azure File Search to extract structured product data from PDF catalogs.

In [33]:
class DataExtractionExecutor(Executor):
    """Extracts product data from PDF files using Azure AI file search."""

    @handler
    async def handle_chat_message(self, message: list[ChatMessage], ctx: WorkflowContext[dict[str, Any]]) -> None:
        """Handle initial data extraction request from DevUI."""
        user_query = str(message[-1]) if message else "Analyze products"
        await self._extract_data(user_query, ctx)
    
    @handler
    async def handle_string(self, message: str, ctx: WorkflowContext[dict[str, Any]]) -> None:
        """Handle direct string input for testing."""
        await self._extract_data(message, ctx)
    
    async def _extract_data(self, query: str, ctx: WorkflowContext[dict[str, Any]]) -> None:
        """Core extraction logic."""
        print("="*70)
        print("🤖 AGENT 1: DOCUMENT SEARCH & DATA EXTRACTION")
        print("="*70)
        print(f"Processing request: {query}")
        
        project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
        client = AzureAIAgentClient(endpoint=project_endpoint, async_credential=AzureCliCredential())
        
        files: List[FileInfo] = []
        vector_store: Optional[VectorStore] = None
        extracted_products = []
        
        try:
            # 1. Find PDF files
            input_path = Path(FOLDERS['input'])
            pdf_files = list(input_path.glob('*.pdf'))
            
            if not pdf_files:
                error_message = f"""
 **No PDF files found!**

Please add PDF catalog files to: `{FOLDERS['input']}`

**Instructions:**
1. Place your product catalog PDFs in the folder above
2. Supported formats: PDF files with product information
3. Expected content: Product names, SKUs, prices, descriptions, features
4. Run this workflow again after adding PDFs

**Example PDF names:**
- haworth-tables-fixed-height_gsa-price-list-part-1.pdf
- knoll-ReffProfilesVolTwo.pdf
- manufacturer-catalog.pdf
"""
                print(error_message)
                
                # Don't use sample data - fail fast and ask user to provide PDFs
                await ctx.send_message({"products": [], "error": error_message})
                return
                
            print(f"\n Found {len(pdf_files)} PDF file(s)")
            for pdf in pdf_files:
                print(f"   - {pdf.name} ({pdf.stat().st_size / 1024 / 1024:.2f} MB)")
            
            # 2. Upload files to Azure AI
            print("\n⬆  Uploading files to Azure AI...")
            file_ids = []
            for pdf_file in pdf_files:
                file = await client.project_client.agents.files.upload_and_poll(
                    file_path=str(pdf_file), purpose="assistants"
                )
                files.append(file)
                file_ids.append(file.id)
                print(f"    {pdf_file.name} → {file.id}")
            
            # 3. Create vector store with the uploaded files
            print("\n Creating vector store for document search...")
            vector_store = await client.project_client.agents.vector_stores.create_and_poll(
                file_ids=file_ids, name="competitive_intelligence_store"
            )
            print(f" Vector store created: {vector_store.id}")
            print(f"   Status: {vector_store.status}")
            print(f"   File count: {vector_store.file_counts.completed}/{vector_store.file_counts.total}")
            
            # 4. Create file search tool pointing to the vector store
            file_search_tool = HostedFileSearchTool(
                inputs=[HostedVectorStoreContent(vector_store_id=vector_store.id)]
            )
            
            # 5. Create AI agent with file search capability
            print("\n🤖 Creating data extraction agent with file search...")
            async with (
                AzureCliCredential() as credential,
                AzureAIAgentClient(
                    endpoint=project_endpoint, 
                    async_credential=credential
                ).create_agent(
                    name="DataExtractionAgent",
                    instructions="""
                    You are a product catalog data extraction specialist with access to PDF documents via file search.
                    
                    Your task: Use the file search tool to find and extract ALL products from the uploaded PDF catalogs.
                    
                    SEARCH STRATEGY:
                    - Search through ALL pages and sections of each PDF
                    - Look for product listings, price lists, SKU numbers, model numbers
                    - Each document may have multiple chunks - search them all
                    - Use multiple search queries if needed to cover all products
                    
                    EXTRACTION FORMAT:
                    For EACH product found, extract as JSON:
                    {
                        "product_name": "Official product name/model",
                        "sku": "Product SKU or model number",
                        "price": numeric_value,
                        "price_text": "Original price text from catalog",
                        "description": "Product description",
                        "features": ["feature1", "feature2", ...],
                        "category": "Product category (e.g., Tables, Chairs, Desks)",
                        "dimensions": "Dimensions if available",
                        "manufacturer": "Manufacturer/brand name (e.g., Haworth, Knoll)",
                        "source_file": "Which PDF this came from"
                    }
                    
                    IMPORTANT REQUIREMENTS:
                    - Extract numeric price only (e.g., from "$1,234.56" extract 1234.56)
                    - Be thorough - aim for 20-30+ products per catalog
                    - Scan through all document chunks
                    - Return ONLY valid JSON array, no additional text or markdown
                    - If you find products across multiple searches, combine them all
                    
                    QUALITY CHECKS:
                    - Verify each product has a name and price
                    - Include SKU/model number whenever available
                    - Capture manufacturer name from the document
                    - Note which PDF file each product came from
                    """,
                    tools=[file_search_tool],
                ) as agent,
            ):
                print(" Agent created with file search capability")
                print("\n Extracting product data from vector store chunks...")
                
                # Run extraction with clear instructions
                response = await agent.run(
                    """Use the file search tool to thoroughly search through ALL uploaded PDF catalogs.
                    
                    Search for:
                    - Product names and model numbers
                    - SKUs and part numbers
                    - Prices and pricing information
                    - Product descriptions and features
                    - Categories (tables, chairs, desks, etc.)
                    - Manufacturer information
                    
                    Extract EVERY product you can find across all document chunks.
                    Return a comprehensive JSON array with ALL products.
                    
                    Format: [{"product_name": "...", "sku": "...", "price": ..., ...}, ...]
                    
                    Be thorough - these are furniture catalogs from manufacturers like Haworth and Knoll."""
                )
                
                print(f"\n Agent response received ({len(response.text)} chars)")
                
                # Parse JSON response
                response_text = response.text
                
                # Clean up markdown code blocks if present
                if "```json" in response_text:
                    response_text = response_text.split("```json")[1].split("```")[0].strip()
                elif "```" in response_text:
                    response_text = response_text.split("```")[1].split("```")[0].strip()
                
                # Try to parse JSON
                try:
                    extracted_products = json.loads(response_text)
                    print(f"\n Successfully parsed {len(extracted_products)} products from JSON")
                except json.JSONDecodeError as e:
                    print(f"\n  JSON parse error: {e}")
                    print("Saving raw response for debugging...")
                    raw_file = Path(FOLDERS['data']) / "extraction_response.txt"
                    with open(raw_file, 'w', encoding='utf-8') as f:
                        f.write(response.text)
                    print(f" Saved raw response to: {raw_file}")
                    print("\nPlease check the response file and verify the PDF content.")
                    extracted_products = []
            
            if extracted_products:
                print(f"\n📦 Extracted {len(extracted_products)} products:")
                print("\n Sample products:")
                for i, product in enumerate(extracted_products[:5], 1):
                    print(f"\n{i}. {product.get('product_name', 'Unknown')}")
                    print(f"   SKU: {product.get('sku', 'N/A')}")
                    print(f"   Price: {product.get('price_text', product.get('price', 'N/A'))}")
                    print(f"   Manufacturer: {product.get('manufacturer', 'N/A')}")
                    print(f"   Category: {product.get('category', 'N/A')}")
                    print(f"   Source: {product.get('source_file', 'N/A')}")
                
                if len(extracted_products) > 5:
                    print(f"\n... and {len(extracted_products) - 5} more products")
            else:
                print("\n  No products extracted - check PDF content and format")
            
            # Save extracted data
            output_file = Path(FOLDERS['data']) / "extracted_products.json"
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(extracted_products, f, indent=2, ensure_ascii=False)
            print(f"\n Saved to: {output_file}")
            
            # Send to next agent
            await ctx.send_message({"products": extracted_products})
            
        except Exception as e:
            print(f"\n Error: {e}")
            import traceback
            traceback.print_exc()
            # Send whatever data we have
            await ctx.send_message({"products": extracted_products, "error": str(e)})
        
        finally:
            # Cleanup resources
            if vector_store:
                try:
                    await client.project_client.agents.vector_stores.delete(vector_store.id)
                    print("\n🧹 Cleaned up vector store")
                except: 
                    pass
            
            for file in files:
                try:
                    await client.project_client.agents.files.delete(file.id)
                except: 
                    pass
            
            if files:
                print(f"🧹 Cleaned up {len(files)} file(s)")
            
            await client.close()
            print("🧹 Closed Azure AI client")

print(" DataExtractionExecutor defined")

 DataExtractionExecutor defined


## Step 5: Build Agent 2 - Pricing Analysis Executor

This executor analyzes pricing strategies and generates insights.

In [34]:
class PricingAnalysisExecutor(Executor):
    """Analyzes pricing data and generates comprehensive insights using AI."""

    @handler
    async def handle_data(self, message: dict[str, Any], ctx: WorkflowContext[dict[str, Any]]) -> None:
        """Handle pricing analysis with AI-powered insights."""
        print("="*70)
        print(" AGENT 2: PRICING ANALYSIS")
        print("="*70)
        
        products = message.get("products", [])
        if not products:
            print("  No products to analyze")
            await ctx.send_message({"analysis": "No data available", "products": []})
            return
        
        print(f"\n Analyzing {len(products)} products...")
        
        project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
        
        try:
            # Prepare data summary for AI analysis
            products_summary = json.dumps(products, indent=2)
            
            # Limit to avoid token issues but provide rich data
            summary_text = products_summary[:15000] if len(products_summary) > 15000 else products_summary
            
            print(f"\n🤖 Creating AI pricing analysis agent...")
            async with (
                AzureCliCredential() as credential,
                AzureAIAgentClient(
                    endpoint=project_endpoint,
                    async_credential=credential
                ).create_agent(
                    name="PricingAnalysisAgent",
                    instructions="""
                    You are a senior pricing strategy analyst specializing in competitive intelligence.
                    
                    Your task: Analyze the provided product pricing data and generate comprehensive insights.
                    
                    Provide detailed analysis on:
                    1. PRICING DISTRIBUTION: Min, max, average, median prices overall and by category
                    2. PRICING STRATEGIES: Identify pricing tiers (budget, mid-range, premium)
                    3. COMPETITIVE POSITIONING: Compare manufacturers/brands on pricing
                    4. PRICE-FEATURE CORRELATION: Analyze if higher prices correlate with more features
                    5. OUTLIERS: Identify unusually expensive or cheap products
                    6. MARKET INSIGHTS: What do the pricing patterns reveal about the market?
                    7. RECOMMENDATIONS: Pricing strategy recommendations based on analysis
                    
                    Return analysis as structured markdown with clear sections and bullet points.
                    Be specific with numbers and examples from the data.
                    """,
                ) as agent,
            ):
                print(" Pricing analysis agent created")
                print("\n Running comprehensive pricing analysis...")
                
                query = f"""
                Analyze the following product pricing data and provide comprehensive insights:
                
                {summary_text}
                
                Provide detailed pricing analysis with:
                - Statistical summary (min, max, average, median, std dev)
                - Pricing tier breakdown
                - Category-based analysis
                - Manufacturer/brand comparison
                - Feature-price correlation insights
                - Market positioning recommendations
                - Strategic pricing recommendations
                """
                
                response = await agent.run(query)
                analysis = response.text
                
                # Save analysis
                output_file = Path(FOLDERS['data']) / "pricing_analysis.md"
                with open(output_file, 'w', encoding='utf-8') as f:
                    f.write(analysis)
                
                print(f"\n AI-powered pricing analysis complete")
                print(f" Saved to: {output_file}")
                print(f"\n📄 Analysis Preview (first 500 chars):")
                print(analysis[:500] + "..." if len(analysis) > 500 else analysis)
        
        except Exception as e:
            print(f"\n  AI analysis error: {e}")
            print("Falling back to basic statistical analysis...")
            
            # Fallback: Basic pandas analysis
            df = pd.DataFrame(products)
            if 'price' in df.columns:
                df['price_numeric'] = pd.to_numeric(df['price'], errors='coerce')
                df = df[df['price_numeric'].notna()]
                
                analysis = f"""## Pricing Analysis (Basic Statistics)

### Summary Statistics
- **Total Products**: {len(df)}
- **Price Range**: ${df['price_numeric'].min():.2f} - ${df['price_numeric'].max():.2f}
- **Average Price**: ${df['price_numeric'].mean():.2f}
- **Median Price**: ${df['price_numeric'].median():.2f}
- **Std Deviation**: ${df['price_numeric'].std():.2f}

### Price Distribution
- **Budget Tier (<${df['price_numeric'].quantile(0.33):.2f})**: {len(df[df['price_numeric'] < df['price_numeric'].quantile(0.33)])} products
- **Mid-Range (${df['price_numeric'].quantile(0.33):.2f} - ${df['price_numeric'].quantile(0.67):.2f})**: {len(df[(df['price_numeric'] >= df['price_numeric'].quantile(0.33)) & (df['price_numeric'] < df['price_numeric'].quantile(0.67))])} products
- **Premium (>${df['price_numeric'].quantile(0.67):.2f})**: {len(df[df['price_numeric'] >= df['price_numeric'].quantile(0.67)])} products
"""
            else:
                analysis = "Price data not available for analysis."
            
            # Save fallback analysis
            output_file = Path(FOLDERS['data']) / "pricing_analysis.md"
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(analysis)
            print(f" Saved basic analysis to: {output_file}")
        
        # Send to next agent
        await ctx.send_message({"analysis": analysis, "products": products})

print(" PricingAnalysisExecutor defined")

 PricingAnalysisExecutor defined


## Step 6: Build Agent 3 - Visualization Generator Executor

This executor creates charts and graphs.

In [35]:
class VisualizationExecutor(Executor):
    """Generates visualizations for the report."""

    @handler
    async def handle_analysis(self, message: dict[str, Any], ctx: WorkflowContext[dict[str, Any]]) -> None:
        """Handle visualization generation."""
        print("="*70)
        print(" AGENT 3: VISUALIZATION GENERATOR")
        print("="*70)
        
        products = message.get("products", [])
        analysis = message.get("analysis", "")
        
        if not products:
            print("  No products for visualization")
            await ctx.send_message({"analysis": analysis, "products": products, "charts": []})
            return
        
        print(f"\n Generating visualizations for {len(products)} products...")
        
        df = pd.DataFrame(products)
        charts = []
        
        try:
            if 'price' in df.columns:
                df['price_numeric'] = pd.to_numeric(df['price'], errors='coerce')
                df = df[df['price_numeric'].notna()]
                
                # Chart 1: Price Distribution
                plt.figure(figsize=(12, 6))
                plt.hist(df['price_numeric'], bins=20, edgecolor='black', alpha=0.7, color='steelblue')
                plt.xlabel('Price ($)', fontsize=12)
                plt.ylabel('Count', fontsize=12)
                plt.title('Price Distribution', fontsize=14, fontweight='bold')
                plt.grid(axis='y', alpha=0.3)
                chart1_path = Path(FOLDERS['charts']) / 'price_distribution.png'
                plt.savefig(chart1_path, dpi=300, bbox_inches='tight')
                plt.close()
                charts.append(('Price Distribution', str(chart1_path)))
                print(f"    {chart1_path.name}")
                
                # Chart 2: Category breakdown (if available)
                if 'category' in df.columns and df['category'].notna().any():
                    plt.figure(figsize=(10, 8))
                    category_counts = df['category'].value_counts()
                    plt.pie(category_counts.values, labels=category_counts.index, 
                           autopct='%1.1f%%', startangle=90, colors=sns.color_palette('Set3'))
                    plt.title('Product Distribution by Category', fontsize=14, fontweight='bold')
                    chart2_path = Path(FOLDERS['charts']) / 'category_distribution.png'
                    plt.savefig(chart2_path, dpi=300, bbox_inches='tight')
                    plt.close()
                    charts.append(('Category Distribution', str(chart2_path)))
                    print(f"    {chart2_path.name}")
        
        except Exception as e:
            print(f"  Visualization error: {e}")
        
        print(f"\n Generated {len(charts)} chart(s)")
        
        # Send to next agent
        await ctx.send_message({
            "analysis": analysis,
            "products": products,
            "charts": charts
        })

print(" VisualizationExecutor defined")

 VisualizationExecutor defined


## Step 7: Build Agent 4 - Report Generator Executor

This executor compiles the final report with all analysis and visualizations.

In [ ]:
class ReportGeneratorExecutor(Executor):
    """Generates comprehensive final report with AI-powered recommendations."""

    @handler
    async def handle_report(self, message: dict[str, Any], ctx: WorkflowContext[list[ChatMessage], list[ChatMessage]]) -> None:
        """Handle comprehensive report generation."""
        print("="*70)
        print(" AGENT 4: COMPREHENSIVE REPORT GENERATOR")
        print("="*70)
        
        analysis = message.get("analysis", "")
        charts = message.get("charts", [])
        products = message.get("products", [])
        
        print(f"\n📄 Generating comprehensive final report...")
        
        # Calculate statistics
        df = pd.DataFrame(products)
        if 'price' in df.columns:
            df['price_numeric'] = pd.to_numeric(df['price'], errors='coerce')
            df = df[df['price_numeric'].notna()]
        
        # Generate timestamp
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        # Build comprehensive report header
        report = f"""# Competitive Intelligence Report

**Generated:** {timestamp}  
**Products Analyzed:** {len(products)}  
**Catalogs Processed:** {len(df['source_file'].unique()) if 'source_file' in df.columns and len(df) > 0 else 'N/A'}

---

## Executive Summary

This report provides a comprehensive competitive intelligence analysis of products from multiple manufacturer catalogs. The analysis includes pricing strategies, feature comparisons, market positioning, and actionable recommendations.

### Key Findings

"""
        
        if len(df) > 0:
            report += f"""- **Total Products Analyzed:** {len(products)}
- **Price Range:** ${df['price_numeric'].min():.2f} - ${df['price_numeric'].max():.2f}
- **Average Price:** ${df['price_numeric'].mean():.2f}
- **Median Price:** ${df['price_numeric'].median():.2f}
- **Categories:** {', '.join(df['category'].unique()[:5]) if 'category' in df.columns else 'N/A'}

"""
        else:
            report += "- Limited product data available for analysis.\n\n"
        
        report += f"""---

## 1. Pricing Analysis

{analysis if analysis else 'Pricing analysis not available.'}

---

## 2. Visual Analysis

"""
        
        # Add charts to report
        for chart_name, chart_path in charts:
            rel_path = Path(chart_path).relative_to(Path(FOLDERS['output']).parent)
            report += f"""###../ {chart_name}

![{chart_name}]({rel_path})

"""
        
        # Add product insights section
        if len(df) > 0:
            report += f"""---

## 3. Product Insights

### Top 10 Most Expensive Products

| Rank | Product | SKU | Price | Category |
|------|---------|-----|-------|----------|
"""
            
            top_products = df.nlargest(min(10, len(df)), 'price_numeric')
            for i, (_, product) in enumerate(top_products.iterrows(), 1):
                report += f"| {i} | {product.get('product_name', 'N/A')} | {product.get('sku', 'N/A')} | ${product['price_numeric']:.2f} | {product.get('category', 'N/A')} |\n"
            
            report += f"""
### Top 10 Most Affordable Products

| Rank | Product | SKU | Price | Category |
|------|---------|-----|-------|----------|
"""
            
            bottom_products = df.nsmallest(min(10, len(df)), 'price_numeric')
            for i, (_, product) in enumerate(bottom_products.iterrows(), 1):
                report += f"| {i} | {product.get('product_name', 'N/A')} | {product.get('sku', 'N/A')} | ${product['price_numeric']:.2f} | {product.get('category', 'N/A')} |\n"
        
        # Generate AI-powered strategic recommendations
        print("\n🤖 Generating AI-powered strategic recommendations...")
        
        project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
        
        try:
            async with (
                AzureCliCredential() as credential,
                AzureAIAgentClient(
                    endpoint=project_endpoint,
                    async_credential=credential
                ).create_agent(
                    name="ReportGeneratorAgent",
                    instructions="""
                    You are a strategic business analyst and competitive intelligence expert.
                    Based on competitive intelligence data, generate actionable strategic recommendations.
                    
                    Focus on:
                    - Pricing strategy optimization
                    - Market positioning opportunities
                    - Competitive advantages
                    - Product portfolio recommendations
                    - Market gap analysis
                    
                    Format response as clear, numbered recommendations with specific insights.
                    Be specific and actionable.
                    """,
                ) as agent,
            ):
                # Prepare summary for recommendations
                if len(df) > 0:
                    summary_data = {
                        "total_products": len(products),
                        "price_range": f"${df['price_numeric'].min():.2f} - ${df['price_numeric'].max():.2f}",
                        "average_price": f"${df['price_numeric'].mean():.2f}",
                        "categories": list(df['category'].unique()[:10]) if 'category' in df.columns else ["Various"],
                        "manufacturers": list(df['manufacturer'].unique()[:10]) if 'manufacturer' in df.columns else ["Various"],
                    }
                else:
                    summary_data = {"total_products": 0, "note": "Limited data"}
                
                query = f"""
                Based on this competitive intelligence summary, provide 5-7 strategic recommendations:
                
                {json.dumps(summary_data, indent=2)}
                
                Provide specific, actionable recommendations for:
                1. Pricing strategy
                2. Product positioning
                3. Market opportunities
                4. Competitive advantages
                5. Portfolio optimization
                """
                
                response = await agent.run(query)
                recommendations = response.text
                
                report += f"""
---

## 4. Strategic Recommendations

{recommendations}

---

## 5. Methodology

This competitive intelligence report was generated using a multi-agent AI workflow:

1. **Document Search Agent:** Extracted product data from PDF catalogs using Azure AI File Search
2. **Data Extraction Agent:** Structured and normalized product information with comprehensive detail extraction
3. **Pricing Analysis Agent:** Performed AI-powered deep analysis of pricing strategies and market positioning
4. **Visualization Agent:** Generated statistical charts and graphs for visual analysis
5. **Report Generator Agent:** Compiled comprehensive analysis with AI-powered strategic recommendations

### Data Sources

"""
                
                if len(df) > 0 and 'source_file' in df.columns:
                    for source in df['source_file'].unique():
                        count = len(df[df['source_file'] == source])
                        report += f"- {source}: {count} products\n"
                else:
                    report += "- Sample data used for demonstration\n"
                
                report += f"""
---

## Appendix: Full Product List

Complete product data available in: `{FOLDERS['data']}/extracted_products.json`

---

*Report generated by Azure AI Agents - Multi-Agent Workflow with DevUI Integration*
"""
        
        except Exception as e:
            print(f"\n  Recommendations generation error: {e}")
            # Add basic recommendations as fallback
            report += f"""
---

## 4. Strategic Recommendations

Based on the competitive analysis:

1. **Price Positioning**: Products show clear price tier segmentation - consider positioning strategy accordingly
2. **Market Coverage**: Analyze distribution across price ranges for portfolio optimization
3. **Feature-Price Optimization**: Evaluate if premium pricing correlates with enhanced features
4. **Competitive Gaps**: Identify underserved price points or product categories
5. **Quality Differentiation**: Use pricing analysis to position quality tiers effectively

---

## 5. Methodology

Multi-agent workflow with data extraction, analysis, visualization, and report generation.

---

*Report generated by Azure AI Agents - Multi-Agent Workflow*
"""
        
        # Save report
        report_path = Path(FOLDERS['output']) / f"competitive_intelligence_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
        with open(report_path, 'w', encoding='utf-8') as f:
            f.write(report)
        
        print(f"\n Comprehensive report generated")
        print(f" Saved to: {report_path}")
        print(f"\n Report includes:")
        print(f"   - Executive summary with key metrics")
        print(f"   - AI-powered detailed pricing analysis")
        print(f"   - {len(charts)} visualization(s)")
        print(f"   - Product insights (top/bottom products)")
        print(f"   - AI-generated strategic recommendations")
        print(f"   - Methodology and data sources")
        
        # Yield final output as ChatMessage for DevUI compatibility
        result = f""" **Comprehensive Competitive Intelligence Report Generated!**

 Analyzed {len(products)} products  
 Created {len(charts)} visualizations  
🤖 AI-powered pricing analysis and strategic recommendations  
📄 Report: `{report_path.name}`

**Key Insights:**
- Price Range: ${df['price_numeric'].min():.2f} - ${df['price_numeric'].max():.2f}
- Average: ${df['price_numeric'].mean():.2f}
- {len(df['category'].unique()) if 'category' in df.columns and len(df) > 0 else 0} categories analyzed

Open the report file to view the complete analysis with AI-powered recommendations.
"""
        
        # Create ChatMessage for DevUI compatibility
        from agent_framework._types import ChatMessage as CM
        result_message = CM(role="assistant", content=result)
        await ctx.yield_output([result_message])

print(" ReportGeneratorExecutor defined")

 ReportGeneratorExecutor defined


## Step 8: Build the Sequential Workflow

Now we connect all 4 agents into a sequential workflow using `SequentialBuilder`.

In [37]:
# Create executor instances
data_extractor = DataExtractionExecutor(id="data_extraction")
pricing_analyzer = PricingAnalysisExecutor(id="pricing_analysis")
viz_generator = VisualizationExecutor(id="visualization_generation")
report_generator = ReportGeneratorExecutor(id="report_generation")

# Build sequential workflow
workflow = (
    SequentialBuilder()
    .participants([
        data_extractor,
        pricing_analyzer,
        viz_generator,
        report_generator,
    ])
    .build()
)

print(" Workflow built successfully!")
print("\n📋 Workflow Steps:")
print("   1. Data Extraction (Azure File Search)")
print("   2. Pricing Analysis")
print("   3. Visualization Generation")
print("   4. Report Generation")

 Workflow built successfully!

📋 Workflow Steps:
   1. Data Extraction (Azure File Search)
   2. Pricing Analysis
   3. Visualization Generation
   4. Report Generation


## Step 9: Run the Workflow

Execute the complete workflow with a query.

In [38]:
# Run the workflow
print("\n" + "="*70)
print(" STARTING COMPETITIVE INTELLIGENCE WORKFLOW")
print("="*70)

initial_query = "Analyze competitive intelligence from product catalogs"

result = await workflow.run(initial_query)

print("\n" + "="*70)
print(" WORKFLOW COMPLETED")
print("="*70)
print(f"\n Result: {result}")
print(f"\n📂 Output Files:")
print(f"   - Data: {FOLDERS['data']}/extracted_products.json")
print(f"   - Analysis: {FOLDERS['data']}/pricing_analysis.md")
print(f"   - Charts: {FOLDERS['charts']}/*.png")
print(f"   - Report: {FOLDERS['output']}/competitive_intelligence_report_*.md")


 STARTING COMPETITIVE INTELLIGENCE WORKFLOW
🤖 AGENT 1: DOCUMENT SEARCH & DATA EXTRACTION
Processing request: <agent_framework._types.ChatMessage object at 0x11c04a810>

 Found 3 PDF file(s)
   - knoll-ReffProfilesVolTwo.pdf (15.99 MB)
   - haworth-tables-fixed-height_gsa-price-list-part-2.pdf (9.02 MB)
   - haworth-tables-fixed-height_gsa-price-list-part-1.pdf (11.54 MB)

⬆  Uploading files to Azure AI...
    knoll-ReffProfilesVolTwo.pdf → assistant-Gcv5qJPMwd8WPRYnoEq3jR
    knoll-ReffProfilesVolTwo.pdf → assistant-Gcv5qJPMwd8WPRYnoEq3jR
    haworth-tables-fixed-height_gsa-price-list-part-2.pdf → assistant-9EtLJ9bSbf4iV74B34eD7b
    haworth-tables-fixed-height_gsa-price-list-part-2.pdf → assistant-9EtLJ9bSbf4iV74B34eD7b
    haworth-tables-fixed-height_gsa-price-list-part-1.pdf → assistant-2TNR5aqWNKYHLezEgzGrGD

 Creating vector store for document search...
    haworth-tables-fixed-height_gsa-price-list-part-1.pdf → assistant-2TNR5aqWNKYHLezEgzGrGD

 Creating vector store for documen